In [2]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_ascertain as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

phases = dr.CLIPS

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [62]:
models = {
    "LGB": LGBMClassifier(),
    # "KNN": KNeighborsClassifier(n_neighbors=5),
    # "LogReg": LogisticRegression(max_iter=1000),
    "RF": RandomForestClassifier(n_estimators=100, max_features="sqrt"),  #  min_samples_split=3
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="error"),
    "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    # train.Metrics.HF_RR, 
    # train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]

# metrics = [
#     train.Metrics.BPM, 
#     train.Metrics.RMSSD, 
#     train.Metrics.HF_RR, 
#     train.Metrics.LF_RR, 
#     train.Metrics.SDNN, 
#     "entropy",
#     "var",
#     "kurtosis",
#     "skewness"
# ]

In [59]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

x, y = train.Train_ASCERTAIN.get_ascertain_data(
    metrics, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)\

acc_results = {
    "LGB": [],
    # "KNN": [],
    # "LogReg": [],
    "RF": [],
    "XGB": [],
    "random": []
}
reports = {
    "LGB": [],
    # "KNN": [],
    # "LogReg": [],
    "RF": [],
    "XGB": [],
    "random": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.train_predict(
        models, x, y, by_subject=True, save_metrics=True, is_resample=False, 
        get_importance=get_importance, drop_subject=False, test_size=0.2, folds=5
    )
    for model_name in acc_results:
        for i in range(len(out[model_name])):
            acc_results[model_name].append(out[model_name][i][0])
            reports[model_name].append(out[model_name][i][1])
        if get_importance:
            try:
                print("")
                # shap.plots.bar(out[model_name][0][2])
                for i in range(len(out[model_name])):
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][i][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
            except Exception as e:
                print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
print("\n")

y_train | y_test:
{1: 1099, 0: 593} | {1: 258, 0: 138}
y_train | y_test:
{1: 1080, 0: 576} | {1: 277, 0: 155}
y_train | y_test:
{1: 1107, 0: 585} | {1: 250, 0: 146}
y_train | y_test:
{1: 1070, 0: 586} | {1: 287, 0: 145}
y_train | y_test:
{1: 1072, 0: 584} | {1: 285, 0: 147}
Fold #0
Model LGB, Predictions: [0 1], [ 67 329]
Feature importance not available for LGB
Fold #1
Model LGB, Predictions: [0 1], [145 287]
Feature importance not available for LGB
Fold #2
Model LGB, Predictions: [0 1], [ 69 327]
Feature importance not available for LGB
Fold #3
Model LGB, Predictions: [0 1], [ 86 346]
Feature importance not available for LGB
Fold #4
Model LGB, Predictions: [0 1], [ 91 341]
Feature importance not available for LGB
Fold #0
Model RF, Predictions: [0 1], [ 53 343]
Fold #1
Model RF, Predictions: [0 1], [117 315]
Fold #2
Model RF, Predictions: [0 1], [ 38 358]
Fold #3
Model RF, Predictions: [0 1], [ 57 375]
Fold #4
Model RF, Predictions: [0 1], [ 73 359]
Fold #0
Model XGB, Predictions: [0 

In [65]:
# TRAIN USING ORIGINAL ASCERTAIN FEATURES
import scipy.io as sio


file = os.path.join(dr.Paths.DATA_DIR, "ASCERTAIN", f"Dt_ECGFeatures.mat")
mat = sio.loadmat(file)["ECGFeatures_58"][0, :]
n_features = mat[0].shape[1]

x = []
for i, clip in enumerate(dr.CLIPS):
    # print(clip + " " + "-"*80)
    for j, s in enumerate(dr.SUBJECTS):
        data = list(mat[j][i])
        data.insert(0, int(s))
        x.append(data)

x = pd.DataFrame(x, columns=["subject"] + [i for i in range(1, n_features+1)])
x = x.drop(list(range(1, 19)), axis=1)

label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

_, y = train.Train_ASCERTAIN.get_ascertain_data(
    metrics, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False
)

inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "LGB": [],
    # "KNN": [],
    # "LogReg": [],
    "RF": [],
    "XGB": [],
    "random": []
}
reports = {
    "LGB": [],
    # "KNN": [],
    # "LogReg": [],
    "RF": [],
    "XGB": [],
    "random": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.train_predict(
        models, x, y, by_subject=True, save_metrics=True, is_resample=False, 
        get_importance=get_importance, drop_subject=False, test_size=0.2, folds=5
    )
    for model_name in acc_results:
        for i in range(len(out[model_name])):
            acc_results[model_name].append(out[model_name][i][0])
            reports[model_name].append(out[model_name][i][1])
        if get_importance:
            try:
                print("")
                # shap.plots.bar(out[model_name][0][2])
                for i in range(len(out[model_name])):
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][i][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
            except Exception as e:
                print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
print("\n")


y_train | y_test:
{1: 1027, 0: 657} | {1: 251, 0: 145}
y_train | y_test:
{1: 1026, 0: 658} | {1: 252, 0: 144}
y_train | y_test:
{1: 1013, 0: 635} | {1: 265, 0: 167}
y_train | y_test:
{1: 1022, 0: 633} | {1: 256, 0: 169}
y_train | y_test:
{1: 1024, 0: 625} | {1: 254, 0: 177}
Fold #0
Model LGB, Predictions: [0 1], [105 291]
Feature importance not available for LGB
Fold #1
Model LGB, Predictions: [0 1], [120 276]
Feature importance not available for LGB
Fold #2
Model LGB, Predictions: [0 1], [106 326]
Feature importance not available for LGB
Fold #3
Model LGB, Predictions: [0 1], [ 99 326]
Feature importance not available for LGB
Fold #4
Model LGB, Predictions: [0 1], [144 287]
Feature importance not available for LGB
Fold #0
Model RF, Predictions: [0 1], [ 58 338]
Fold #1
Model RF, Predictions: [0 1], [ 85 311]
Fold #2
Model RF, Predictions: [0 1], [ 86 346]
Fold #3
Model RF, Predictions: [0 1], [ 73 352]
Fold #4
Model RF, Predictions: [0 1], [105 326]
Fold #0
Model XGB, Predictions: [0 